In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-with-disaster-tweets-cleaning-data/test_data_cleaning.csv
/kaggle/input/nlp-with-disaster-tweets-cleaning-data/train_data_cleaning.csv
/kaggle/input/nlp-with-disaster-tweets-cleaning-data/test_data_cleaning2.csv
/kaggle/input/nlp-with-disaster-tweets-cleaning-data/train_data_cleaning2.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import re
from sklearn import feature_extraction, linear_model, model_selection, metrics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# reading train and test files
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [5]:
# displaying the 5 first columns of train_df
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# split the data into train, test and validation sets
X_train, X_temp, y_train, y_temp = model_selection.train_test_split(train_df["text"], train_df["target"], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
!pip install tweet-preprocessor

In [8]:
!pip install gensim

In [9]:
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords
def preprocess_tweet(text):
    text = remove_stopwords(text)
    text = text.lower()
    text = re.sub('[^\w\s]','',text)
    return text

In [10]:
# text data preprocessing
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def preprocessing(X):
    max_length = max(len(text) for text in X)
    input_ids = []
    attention_masks = []
    for text in X:
        text = preprocess_tweet(text)
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        input_ids.append(encoded_dict["input_ids"])
        attention_masks.append(encoded_dict["attention_mask"])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks

train_input_ids, train_attention_masks = preprocessing(X_train)
validation_input_ids, validation_attention_masks = preprocessing(X_val)

In [11]:
train_labels = torch.tensor(y_train.values)
validation_labels = torch.tensor(y_val.values)

In [12]:
# create a data loader
def create_data_loader(input_ids, attention_masks, labels):
    dataset = TensorDataset(input_ids, attention_masks, labels)
    batch_size = 32
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return train_loader

train_loader = create_data_loader(train_input_ids, train_attention_masks, train_labels)
validation_loader = create_data_loader(validation_input_ids, validation_attention_masks, validation_labels)

In [13]:
# define model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(256, 100),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(100, 50),
    nn.ReLU(),
    nn.Linear(50, 2),
)

# freeze parameters
for param in model.bert.parameters():
    param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# defining loss function and optimizer
from transformers import get_linear_schedule_with_warmup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)
total_steps = len(train_loader) * 6
# Create a learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [17]:
# fine tune the model
num_epochs = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_losses, valid_losses = [], []
valid_loss_min = np.Inf

for epoch in range(num_epochs):
    train_loss, valid_loss = 0 , 0
    model.train()
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2]
        }
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, inputs["labels"])
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs["labels"].size(0)
    
    model.eval()
    for batch in validation_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2]
        }
        with torch.no_grad():
              output = model(**inputs)
        loss = criterion(output.logits,inputs["labels"])
        valid_loss += loss.item() * inputs["labels"].size(0)

    train_loss /= len(train_loader.sampler)
    valid_loss /= len(validation_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print('epoch: {} \ttraining Loss: {:.6f} \tvalidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))

    if valid_loss <= valid_loss_min:
        print('validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

epoch: 1 	training Loss: 0.565733 	validation Loss: 0.539589
validation loss decreased (inf --> 0.539589).  Saving model ...
epoch: 2 	training Loss: 0.569161 	validation Loss: 0.539589
validation loss decreased (0.539589 --> 0.539589).  Saving model ...
epoch: 3 	training Loss: 0.561218 	validation Loss: 0.539589
epoch: 4 	training Loss: 0.563292 	validation Loss: 0.539589
epoch: 5 	training Loss: 0.565949 	validation Loss: 0.539589
epoch: 6 	training Loss: 0.562666 	validation Loss: 0.539589


In [16]:
# testing the model and evaluating it using F1 score
test_input_ids, test_attention_masks = preprocessing(X_test)
test_labels = torch.tensor(y_test.values)
test_loader = create_data_loader(test_input_ids, test_attention_masks, test_labels)

true_labels = []
predicted_labels = []

model.load_state_dict(torch.load('model.pt', map_location=device))

model.eval()
for batch in test_loader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {
        "input_ids": batch[0],
        "attention_mask": batch[1],
        "labels": batch[2]
    }
    with torch.no_grad():
        output = model(**inputs)
    _, predicted = torch.max(output.logits, 1)
    true_labels.extend(inputs["labels"].cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())



In [ ]:
print(len(true_labels))
print(predicted_labels)

In [59]:
f1 = metrics.f1_score(true_labels, predicted_labels)
print(f"F1 Score: {f1}")

F1 Score: 0.7364016736401673
